In [36]:
import pandas as pd
import numpy as np

In [132]:
import matplotlib.pyplot as plt
%matplotlib inline

In [208]:
from coldstart import expand_factors, similar_rest, similar_cat, find_k_similar

In [35]:
from train_model import str_to_l

In [27]:
biz_df = pd.read_csv('biz_with_F.csv')

In [29]:
biz_df['categories'] = biz_df['categories'].map(str_to_l)
biz_df['rat_to_rev'] = biz_df['stars']/biz_df['review_count']

In [8]:
biz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57173 entries, 0 to 57172
Data columns (total 9 columns):
Unnamed: 0                57173 non-null int64
RestaurantsPriceRange2    51111 non-null float64
business_id               57173 non-null object
stars                     57173 non-null float64
review_count              57173 non-null int64
categories                57173 non-null object
biz_num                   57173 non-null int64
id                        53931 non-null float64
features                  53931 non-null object
dtypes: float64(3), int64(3), object(3)
memory usage: 3.9+ MB


In [ ]:
### Заполнить цену во всем треннинг сете и во всем датасете

In [50]:
biz_df['RestaurantsPriceRange2'] = biz_df['RestaurantsPriceRange2'].fillna(2)

In [52]:
test = biz_df[pd.isnull(biz_df.id)]
train = biz_df[~pd.isnull(biz_df.id)]

test.shape, train.shape

((3242, 10), (53931, 10))

'[-0.49883636832237244, 0.11051841080188751, 1.0938549041748047, 1.7710814476013184, -0.05432644858956337, -0.7591550946235657]'

In [53]:
len(train[pd.isnull(train.RestaurantsPriceRange2)])

0

In [54]:
len(test[pd.isnull(test.RestaurantsPriceRange2)])

0

In [55]:
train.describe()

,Unnamed: 0,RestaurantsPriceRange2,stars,review_count,biz_num,id,rat_to_rev
count,53931.000000,53931.000000,53931.000000,53931.000000,5.393100e+04,5.393100e+04,53931.000000
mean,28588.182326,1.703807,3.448193,67.456398,-3.164461e+06,-3.164461e+06,0.281167
std,16492.599579,0.601448,0.781084,166.800347,1.242820e+09,1.242820e+09,0.327242
min,0.000000,1.000000,1.000000,3.000000,-2.147478e+09,-2.147478e+09,0.000502
25%,14316.500000,1.000000,3.000000,8.000000,-1.086039e+09,-1.086039e+09,0.054054
50%,28591.000000,2.000000,3.500000,22.000000,-4.163377e+06,-4.163377e+06,0.148148
75%,42863.500000,2.000000,4.000000,63.000000,1.072152e+09,1.072152e+09,0.388889
max,57172.000000,4.000000,5.000000,7968.000000,2.147459e+09,2.147459e+09,1.666667


In [56]:
train.categories[4]

['Event Planning & Services', 'Caterers', 'Restaurants', 'Breakfast & Brunch']

In [57]:
train.iloc[3333]

Unnamed: 0                                                             3538
RestaurantsPriceRange2                                                    2
business_id                                          MMh9G9Mg9iLEeUzmg7Lyqw
stars                                                                     4
review_count                                                             23
categories                        [Food Trucks, Food, Restaurants, Mexican]
biz_num                                                           502158358
id                                                              5.02158e+08
features                  [0.3375687599182129, -0.0874842181801796, 1.40...
rat_to_rev                                                         0.173913
Name: 3538, dtype: object

In [58]:
test.iloc[222]

Unnamed: 0                                  3851
RestaurantsPriceRange2                         2
business_id               25eFMLlnsrZVLoKkqpjpCg
stars                                          4
review_count                                   4
categories                 [Korean, Restaurants]
biz_num                                944332333
id                                           NaN
features                                     NaN
rat_to_rev                                     1
Name: 3851, dtype: object

In [89]:
r1 = train.iloc[3333]
r2 = test.iloc[223]
similar_rest(r1,r2)


0.7880434782608696

In [144]:
%%time
x = train.apply(similar_rest, axis = 1,r2= r2)

CPU times: user 7.29 s, sys: 87.2 ms, total: 7.38 s
Wall time: 7.38 s


In [145]:
x.describe()

count    53931.000000
mean         1.222898
std          0.355776
min          0.200000
25%          0.958333
50%          1.208333
75%          1.458333
max          3.291667
dtype: float64

In [154]:
(x >np.percentile(x, 95)).sum()

2697

In [163]:
a =(train['features'].iloc[0])
a

'[0.4572981894016266, 0.46191635727882385, 0.4901009202003479, 1.9467405080795288, 1.0112497806549072, 0.0902012512087822]'

In [167]:
%%time
b = train['features'].map(lambda x: x[1:-1]).str.split(',')


CPU times: user 181 ms, sys: 95.3 ms, total: 276 ms
Wall time: 276 ms


In [172]:
b.head(), type(b[33]), type(b[33]), len(b[33]), type(b[33][0])

(0    [0.4572981894016266,  0.46191635727882385,  0....
 1    [0.32101115584373474,  -0.6140308976173401,  0...
 2    [-0.7168925404548645,  0.23761771619319916,  0...
 3    [-0.337391197681427,  0.7905454039573669,  1.5...
 4    [-1.165808081626892,  0.21498383581638336,  0....
 Name: features, dtype: object, list, list, 6, str)

In [180]:
train_expand = expand_factors(train)
feat_c = train_expand.columns[-6:]

In [195]:
len(train_expand[feat_c][(x >np.percentile(x, 98.5))])

800

In [186]:
train_expand[feat_c][(x >np.percentile(x, 95))].mean(axis = 0)

feature_0   -0.164966
feature_1   -0.064750
feature_2    0.852187
feature_3    0.389767
feature_4   -0.756901
feature_5   -0.170802
dtype: float64

In [189]:
len(look) -295042

-1373

In [198]:
train_expand.columns

Index(['Unnamed: 0', 'RestaurantsPriceRange2', 'business_id', 'stars',
       'review_count', 'categories', 'biz_num', 'id', 'rat_to_rev',
       'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5'],
      dtype='object')

In [204]:
X_test = test[:3].copy()

In [205]:
for col in feat_c:
    X_test[col] = 0

In [212]:
Y =X_test.copy()
Y.index

Int64Index([7, 25, 50], dtype='int64')

In [215]:
Y.loc[7,:] = find_k_similar(train_expand, Y.loc[7,:])

In [216]:
Y.loc[7,:]

Unnamed: 0                      NaN
RestaurantsPriceRange2          NaN
business_id                     NaN
stars                           NaN
review_count                    NaN
categories                      NaN
biz_num                         NaN
id                              NaN
features                        NaN
rat_to_rev                      NaN
feature_0                -0.0298472
feature_1                -0.0611764
feature_2                  0.690563
feature_3                 0.0637666
feature_4                 -0.679339
feature_5                 -0.153558
Name: 7, dtype: object